In [1]:
import os
import subprocess
import json
import sys


In [6]:
max_commit = 100 # int(sys.argv[1]) 

# Raw 문자열로 처리하기 위해 문자열 앞에 'r'을 붙임
with open('git-log.json', 'r', encoding='utf-8') as f:
    data = f.read()
    raw_data = r'' + data

# Raw 문자열을 JSON으로 디코드
try:
    json_data = json.loads(raw_data) 
    print(json.dumps(json_data, indent = '\t'))
    
except json.JSONDecodeError as e:
    print(f"JSON 디코드 오류: {e}")

subprocess.run(['mkdir', 'commits'])

[
	{
		"commitHash": "95ca43034fe661556f56dee7c39b6d7b784a0a2e",
		"parent": "c1c7dff2ad082a110aaab3c2c9dc1678773f1a92",
		"subject": "Change default handoffConditionTimeout to 15 minutes. (#14539)",
		"sanitized_subject_line": "Change-default-handoffConditionTimeout-to-15-minutes.-14539",
		"commit_notes": "",
		"body": "* Change default handoffConditionTimeout to 15 minutes.  Most of the time, when handoff is taking this long, it's because something is preventing Historicals from loading new data. In this case, we have two choices:  1) Stop making progress on ingestion, wait for Historicals to load stuff,    and keep the waiting-for-handoff segments available on realtime tasks.    (handoffConditionTimeout = 0, the current default)  2) Continue making progress on ingestion, by exiting the realtime tasks    that were waiting for handoff. Once the Historicals get their act    together, the segments will be loaded, as they are still there on    deep storage. They will just not be continu

CompletedProcess(args=['mkdir', 'commits'], returncode=1)

In [7]:
def read_checkout_file(file):
    try:
        with open(file, 'r', encoding='utf-8') as f:
            content = f.read()
    except:
        print(f"File Open 오류:")
        content = ''
    return content

def write_checkout_file(file_path, content):
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
    except:
        print(f"File Write 오류")

In [9]:
cnt = 0

for commit in json_data:        
    files = []
    commit['changed_file_list'] = []
    
    if cnt >= max_commit:
        break
    
    changed_file_list = subprocess.run(['git', 'show', commit['commitHash'], '--name-only', '--pretty=format:%b'], capture_output=True, text=True)
    if changed_file_list.returncode == 0 and changed_file_list.stdout:
        lines = changed_file_list.stdout.split('\n')
        # Process the rest of the code using the 'lines' variable
    else:
        # Handle the case when the Git command didn't execute successfully
        print("Failed to get the changed file list.")
    
    for line in lines:
        if line.strip() in commit['body']:
            continue
        elif line.strip() != '':
            files.append(line.strip())
            
    is_all_java = all(file.endswith('.java') for file in files)
    
    if is_all_java and files != []:
        diff = subprocess.run(['git', 'show', commit['commitHash'], '-p'], capture_output=True, text=True)
        commit['changed_file_list'] = files
        print(files)
        cnt += 1
        print(commit['commitHash'])
        print(cnt, ":", commit['changed_file_list'])
        
        file_path = os.path.join('commits', f'{cnt}_diff.txt')
        write_checkout_file(file_path, diff.stdout)

['extensions-core/kafka-indexing-service/src/test/java/org/apache/druid/indexing/kafka/KafkaIndexTaskTuningConfigTest.java', 'extensions-core/kafka-indexing-service/src/test/java/org/apache/druid/indexing/kafka/supervisor/KafkaSupervisorTuningConfigTest.java', 'extensions-core/kinesis-indexing-service/src/test/java/org/apache/druid/indexing/kinesis/KinesisIndexTaskTuningConfigTest.java', 'extensions-core/kinesis-indexing-service/src/test/java/org/apache/druid/indexing/kinesis/supervisor/KinesisSupervisorTuningConfigTest.java', 'server/src/main/java/org/apache/druid/segment/indexing/RealtimeTuningConfig.java', 'server/src/test/java/org/apache/druid/segment/indexing/RealtimeTuningConfigTest.java']
95ca43034fe661556f56dee7c39b6d7b784a0a2e
1 : ['extensions-core/kafka-indexing-service/src/test/java/org/apache/druid/indexing/kafka/KafkaIndexTaskTuningConfigTest.java', 'extensions-core/kafka-indexing-service/src/test/java/org/apache/druid/indexing/kafka/supervisor/KafkaSupervisorTuningConfigT

['processing/src/main/java/org/apache/druid/segment/data/BlockLayoutColumnarDoublesSupplier.java', 'processing/src/main/java/org/apache/druid/segment/data/BlockLayoutColumnarFloatsSupplier.java', 'processing/src/main/java/org/apache/druid/segment/data/BlockLayoutColumnarLongsSupplier.java', 'processing/src/main/java/org/apache/druid/segment/data/CompressedColumnarIntsSupplier.java', 'processing/src/main/java/org/apache/druid/segment/data/CompressedVSizeColumnarIntsSupplier.java', 'processing/src/main/java/org/apache/druid/segment/data/DecompressingByteBufferObjectStrategy.java', 'processing/src/main/java/org/apache/druid/segment/data/GenericIndexed.java']
3711c0d987213a0086053916a23775768f6f0a6b
6 : ['processing/src/main/java/org/apache/druid/segment/data/BlockLayoutColumnarDoublesSupplier.java', 'processing/src/main/java/org/apache/druid/segment/data/BlockLayoutColumnarFloatsSupplier.java', 'processing/src/main/java/org/apache/druid/segment/data/BlockLayoutColumnarLongsSupplier.java',

['indexing-service/src/main/java/org/apache/druid/indexing/overlord/Stats.java', 'indexing-service/src/main/java/org/apache/druid/indexing/overlord/TaskMaster.java', 'indexing-service/src/main/java/org/apache/druid/indexing/overlord/TaskQueue.java', 'indexing-service/src/main/java/org/apache/druid/indexing/overlord/config/TaskQueueConfig.java', 'indexing-service/src/test/java/org/apache/druid/indexing/overlord/TaskLockConfigTest.java', 'indexing-service/src/test/java/org/apache/druid/indexing/overlord/TaskQueueScaleTest.java', 'indexing-service/src/test/java/org/apache/druid/indexing/overlord/TaskQueueTest.java', 'indexing-service/src/test/java/org/apache/druid/indexing/overlord/http/OverlordTest.java', 'server/src/main/java/org/apache/druid/server/coordinator/balancer/RandomBalancerStrategy.java', 'server/src/main/java/org/apache/druid/server/coordinator/stats/CoordinatorRunStats.java', 'server/src/main/java/org/apache/druid/server/metrics/TaskCountStatsMonitor.java', 'server/src/main

['extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/exec/ControllerImpl.java', 'extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/indexing/MSQControllerTask.java', 'sql/src/main/java/org/apache/druid/sql/calcite/run/NativeQueryMaker.java', 'sql/src/main/java/org/apache/druid/sql/calcite/run/SqlResults.java', 'sql/src/test/java/org/apache/druid/sql/calcite/CalciteScanSignatureTest.java', 'sql/src/test/java/org/apache/druid/sql/calcite/run/SqlResultsTest.java']
2d5b27358e0a1abefcc3c7ddeb3af613ddf88df5
26 : ['extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/exec/ControllerImpl.java', 'extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/indexing/MSQControllerTask.java', 'sql/src/main/java/org/apache/druid/sql/calcite/run/NativeQueryMaker.java', 'sql/src/main/java/org/apache/druid/sql/calcite/run/SqlResults.java', 'sql/src/test/java/org/apache/druid/sql/calcite/CalciteScanSignatureTest.java', 'sql/src/test/java/org/a

Exception in thread Thread-670:
Traceback (most recent call last):
  File "c:\python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "c:\python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "c:\python39\lib\subprocess.py", line 1479, in _readerthread
    buffer.append(fh.read())
UnicodeDecodeError: 'cp949' codec can't decode byte 0xe9 in position 48744: illegal multibyte sequence


...

['extensions-contrib/kubernetes-overlord-extensions/src/main/java/org/apache/druid/k8s/overlord/KubernetesOverlordModule.java', 'extensions-contrib/kubernetes-overlord-extensions/src/main/java/org/apache/druid/k8s/overlord/KubernetesPeonLifecycle.java', 'extensions-contrib/kubernetes-overlord-extensions/src/main/java/org/apache/druid/k8s/overlord/KubernetesTaskRunnerFactory.java', 'extensions-contrib/kubernetes-overlord-extensions/src/main/java/org/apache/druid/k8s/overlord/KubernetesWorkItem.java', 'extensions-contrib/kubernetes-overlord-extensions/src/main/java/org/apache/druid/k8s/overlord/common/DruidKubernetesClient.java', 'extensions-contrib/kubernetes-overlord-extensions/src/main/java/org/apache/druid/k8s/overlord/common/KubernetesPeonClient.java', 'extensions-contrib/kubernetes-overlord-extensions/src/main/java/org/apache/druid/k8s/overlord/common/LogWatchInputStream.java', 'extensions-contrib/kubernetes-overlord-extensions/src/test/java/org/apache/druid/k8s/overlord/Kubernetes

Exception in thread Thread-1080:
Traceback (most recent call last):
  File "c:\python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "c:\python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "c:\python39\lib\subprocess.py", line 1479, in _readerthread
    buffer.append(fh.read())
UnicodeDecodeError: 'cp949' codec can't decode byte 0xe2 in position 225: illegal multibyte sequence


File Write 오류
['processing/src/main/java/org/apache/druid/segment/join/JoinableFactoryWrapper.java', 'processing/src/test/java/org/apache/druid/segment/join/JoinableFactoryWrapperTest.java']
96a3c00754223626ff1df3799522277a086ec0a1
85 : ['processing/src/main/java/org/apache/druid/segment/join/JoinableFactoryWrapper.java', 'processing/src/test/java/org/apache/druid/segment/join/JoinableFactoryWrapperTest.java']
['indexing-service/src/main/java/org/apache/druid/indexing/common/TaskLockType.java', 'indexing-service/src/main/java/org/apache/druid/indexing/overlord/TaskLockbox.java', 'indexing-service/src/test/java/org/apache/druid/indexing/overlord/TaskLockboxTest.java']
e9913abbbfed82c11d1f876f30f24a768267ab48
86 : ['indexing-service/src/main/java/org/apache/druid/indexing/common/TaskLockType.java', 'indexing-service/src/main/java/org/apache/druid/indexing/overlord/TaskLockbox.java', 'indexing-service/src/test/java/org/apache/druid/indexing/overlord/TaskLockboxTest.java']
['extensions-cor

['processing/src/main/java/org/apache/druid/segment/virtual/NestedFieldVirtualColumn.java', 'processing/src/test/java/org/apache/druid/query/NestedDataTestUtils.java', 'processing/src/test/java/org/apache/druid/query/groupby/NestedDataGroupByQueryTest.java']
aaaff747409729ce11dc81e2de75eb6faa0fed76
92 : ['processing/src/main/java/org/apache/druid/segment/virtual/NestedFieldVirtualColumn.java', 'processing/src/test/java/org/apache/druid/query/NestedDataTestUtils.java', 'processing/src/test/java/org/apache/druid/query/groupby/NestedDataGroupByQueryTest.java']
['extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/exec/ControllerImpl.java', 'extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/input/external/ExternalInputSliceReader.java', 'extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/util/DimensionSchemaUtils.java', 'extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/util/MultiStageQueryContext.java', 'extensions-c

['extensions-contrib/graphite-emitter/src/main/java/org/apache/druid/emitter/graphite/DruidToGraphiteEventConverter.java']
3c62c00d4c86ff002480a28c917afd4bc8dd3451
96 : ['extensions-contrib/graphite-emitter/src/main/java/org/apache/druid/emitter/graphite/DruidToGraphiteEventConverter.java']
['processing/src/test/java/org/apache/druid/query/groupby/GroupByQueryRunnerTest.java', 'server/src/main/java/org/apache/druid/server/QueryScheduler.java', 'server/src/test/java/org/apache/druid/server/QueryResourceTest.java', 'server/src/test/java/org/apache/druid/server/QuerySchedulerTest.java']
a7a4bfd331e320d8925e6f39fdc4861319e5b8bd
97 : ['processing/src/test/java/org/apache/druid/query/groupby/GroupByQueryRunnerTest.java', 'server/src/main/java/org/apache/druid/server/QueryScheduler.java', 'server/src/test/java/org/apache/druid/server/QueryResourceTest.java', 'server/src/test/java/org/apache/druid/server/QuerySchedulerTest.java']
['sql/src/main/java/org/apache/druid/sql/calcite/expression/buil

In [ ]:
cnt = 0

for commit in json_data:
    if cnt > max_commit:
        break
    elif commit['changed_file_list'] != []:
        cnt += 1
        print(cnt)
    else:
        continue
        
    for file in commit['changed_file_list']:
        print(file)
        
        subprocess.run(['git', 'checkout', commit['commitHash'], '--', file])
        after_content = read_checkout_file(file) 
        file_path = os.path.join('commits', f'{cnt}_after_{os.path.basename(file)}')
        write_checkout_file(file_path, after_content)
        
        subprocess.run(['git', 'checkout', commit['parent'], '--', file])
        before_content = read_checkout_file(file)
        file_path = os.path.join('commits', f'{cnt}_before_{os.path.basename(file)}')
        write_checkout_file(file_path, before_content)
        
        subprocess.run(['git', 'checkout', 'HEAD', '--', file])

1
extensions-core/kafka-indexing-service/src/test/java/org/apache/druid/indexing/kafka/KafkaIndexTaskTuningConfigTest.java
extensions-core/kafka-indexing-service/src/test/java/org/apache/druid/indexing/kafka/supervisor/KafkaSupervisorTuningConfigTest.java
extensions-core/kinesis-indexing-service/src/test/java/org/apache/druid/indexing/kinesis/KinesisIndexTaskTuningConfigTest.java
extensions-core/kinesis-indexing-service/src/test/java/org/apache/druid/indexing/kinesis/supervisor/KinesisSupervisorTuningConfigTest.java
server/src/main/java/org/apache/druid/segment/indexing/RealtimeTuningConfig.java
server/src/test/java/org/apache/druid/segment/indexing/RealtimeTuningConfigTest.java
2
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/sql/MSQTaskQueryMaker.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/sql/MSQTaskSqlEngine.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/sql/resources/SqlStatementResource.java
extensions

extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/indexing/client/IndexerControllerClient.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/indexing/client/IndexerWorkerClient.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/indexing/client/IndexerWorkerManagerClient.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/indexing/client/WorkerChatHandler.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/indexing/destination/DataSourceMSQDestination.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/indexing/destination/DurableStorageMSQDestination.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/indexing/destination/MSQDestination.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/indexing/destination/MSQSelectDestination.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/indexing/des

server/src/test/java/org/apache/druid/segment/indexing/granularity/UniformGranularityTest.java
22
sql/src/main/java/org/apache/druid/sql/calcite/external/ExternalOperatorConversion.java
sql/src/test/java/org/apache/druid/sql/calcite/CalciteInsertDmlTest.java
23
sql/src/main/java/org/apache/druid/sql/calcite/expression/builtin/RegexpExtractOperatorConversion.java
sql/src/test/java/org/apache/druid/sql/calcite/CalciteQueryTest.java
24
indexing-service/src/main/java/org/apache/druid/indexing/seekablestream/supervisor/SeekableStreamSupervisor.java
indexing-service/src/test/java/org/apache/druid/indexing/seekablestream/supervisor/SeekableStreamSupervisorStateTest.java
25
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/sql/entity/PageInformation.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/sql/entity/ResultSetInformation.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/sql/resources/SqlStatementResource.java
extension

extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/indexing/MSQSelectDestination.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/indexing/report/MSQResultsReport.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/util/MultiStageQueryContext.java
extensions-core/multi-stage-query/src/test/java/org/apache/druid/msq/exec/MSQSelectTest.java
extensions-core/multi-stage-query/src/test/java/org/apache/druid/msq/indexing/report/MSQTaskReportTest.java
extensions-core/multi-stage-query/src/test/java/org/apache/druid/msq/test/MSQTestBase.java
extensions-core/multi-stage-query/src/test/java/org/apache/druid/msq/util/MultiStageQueryContextTest.java
integration-tests/src/main/java/org/apache/druid/testing/utils/MsqTestQueryHelper.java
36
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/guice/SqlTaskModule.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/indexing/MSQControllerTask.java
exte

extensions-core/datasketches/src/test/java/org/apache/druid/query/aggregation/datasketches/quantiles/sql/DoublesSketchSqlAggregatorTest.java
extensions-core/testing-tools/src/main/java/org/apache/druid/query/sql/SleepOperatorConversion.java
integration-tests-ex/tools/src/main/java/org/apache/druid/testing/tools/SleepOperatorConversion.java
sql/src/main/java/org/apache/druid/sql/calcite/expression/OperatorConversions.java
sql/src/main/java/org/apache/druid/sql/calcite/expression/builtin/BTrimOperatorConversion.java
sql/src/main/java/org/apache/druid/sql/calcite/expression/builtin/ContainsOperatorConversion.java
sql/src/main/java/org/apache/druid/sql/calcite/expression/builtin/DateTruncOperatorConversion.java
sql/src/main/java/org/apache/druid/sql/calcite/expression/builtin/LPadOperatorConversion.java
sql/src/main/java/org/apache/druid/sql/calcite/expression/builtin/LTrimOperatorConversion.java
sql/src/main/java/org/apache/druid/sql/calcite/expression/builtin/ParseLongOperatorConversion.

server/src/main/java/org/apache/druid/server/coordinator/balancer/CachingCostBalancerStrategyFactory.java
server/src/main/java/org/apache/druid/server/coordinator/balancer/ClusterCostCache.java
server/src/main/java/org/apache/druid/server/coordinator/balancer/CostBalancerStrategy.java
server/src/main/java/org/apache/druid/server/coordinator/balancer/CostBalancerStrategyFactory.java
server/src/main/java/org/apache/druid/server/coordinator/balancer/DiskNormalizedCostBalancerStrategy.java
server/src/main/java/org/apache/druid/server/coordinator/balancer/DiskNormalizedCostBalancerStrategyFactory.java
server/src/main/java/org/apache/druid/server/coordinator/balancer/RandomBalancerStrategy.java
server/src/main/java/org/apache/druid/server/coordinator/balancer/RandomBalancerStrategyFactory.java
server/src/main/java/org/apache/druid/server/coordinator/balancer/ReservoirSegmentSampler.java
server/src/main/java/org/apache/druid/server/coordinator/balancer/SegmentsCostCache.java
server/src/main/j

server/src/test/java/org/apache/druid/server/coordinator/simulate/CoordinatorSimulationBaseTest.java
server/src/test/java/org/apache/druid/server/coordinator/simulate/CoordinatorSimulationBuilder.java
server/src/test/java/org/apache/druid/server/coordinator/simulate/RoundRobinAssignmentTest.java
server/src/test/java/org/apache/druid/server/coordinator/simulate/SegmentBalancingTest.java
server/src/test/java/org/apache/druid/server/coordinator/simulate/SegmentLoadingNegativeTest.java
File Open 오류:
server/src/test/java/org/apache/druid/server/coordinator/simulate/SegmentLoadingTest.java
server/src/test/java/org/apache/druid/server/coordinator/simulate/TestSegmentsMetadataManager.java
server/src/test/java/org/apache/druid/server/http/CoordinatorDynamicConfigTest.java
server/src/test/java/org/apache/druid/server/http/DataSourcesResourceTest.java
server/src/test/java/org/apache/druid/server/http/MetadataResourceTest.java
services/src/main/java/org/apache/druid/cli/CliCoordinator.java
53
exte

extensions-core/kafka-indexing-service/src/main/java/org/apache/druid/indexing/kafka/KafkaSamplerSpec.java
extensions-core/kafka-indexing-service/src/test/java/org/apache/druid/indexing/kafka/KafkaIndexTaskTest.java
extensions-core/kafka-indexing-service/src/test/java/org/apache/druid/indexing/kafka/KafkaSamplerSpecTest.java
extensions-core/kinesis-indexing-service/src/main/java/org/apache/druid/indexing/kinesis/KinesisIndexTask.java
extensions-core/kinesis-indexing-service/src/main/java/org/apache/druid/indexing/kinesis/KinesisIndexingServiceModule.java
extensions-core/kinesis-indexing-service/src/main/java/org/apache/druid/indexing/kinesis/KinesisSamplerSpec.java
extensions-core/kinesis-indexing-service/src/test/java/org/apache/druid/indexing/kinesis/KinesisIndexTaskSerdeTest.java
extensions-core/kinesis-indexing-service/src/test/java/org/apache/druid/indexing/kinesis/KinesisSamplerSpecTest.java
extensions-core/multi-stage-query/src/main/java/org/apache/druid/msq/input/NilInputSource

File Open 오류:
extensions-contrib/kubernetes-overlord-extensions/src/test/java/org/apache/druid/k8s/overlord/KubernetesPeonLifecycleTest.java
extensions-contrib/kubernetes-overlord-extensions/src/test/java/org/apache/druid/k8s/overlord/KubernetesTaskRunnerFactoryTest.java
extensions-contrib/kubernetes-overlord-extensions/src/test/java/org/apache/druid/k8s/overlord/KubernetesWorkItemTest.java
extensions-contrib/kubernetes-overlord-extensions/src/test/java/org/apache/druid/k8s/overlord/common/KubernetesPeonClientTest.java
extensions-contrib/kubernetes-overlord-extensions/src/test/java/org/apache/druid/k8s/overlord/common/LogWatchInputStreamTest.java
File Open 오류:
83
server/src/main/java/org/apache/druid/server/security/PreResponseAuthorizationCheckFilter.java
server/src/test/java/org/apache/druid/server/http/security/PreResponseAuthorizationCheckFilterTest.java
84
processing/src/main/java/org/apache/druid/query/metadata/SegmentAnalyzer.java
processing/src/main/java/org/apache/druid/query/

processing/src/main/java/org/apache/druid/segment/nested/NestedDataColumnSerializer.java
processing/src/main/java/org/apache/druid/segment/nested/NestedDataColumnV5.java
processing/src/main/java/org/apache/druid/segment/nested/NestedDataComplexColumn.java
processing/src/main/java/org/apache/druid/segment/nested/ScalarDoubleColumn.java
processing/src/main/java/org/apache/druid/segment/nested/ScalarDoubleColumnAndIndexSupplier.java
processing/src/main/java/org/apache/druid/segment/nested/ScalarDoubleColumnSerializer.java
processing/src/main/java/org/apache/druid/segment/nested/ScalarDoubleFieldColumnWriter.java
processing/src/main/java/org/apache/druid/segment/nested/ScalarLongColumn.java
processing/src/main/java/org/apache/druid/segment/nested/ScalarLongColumnAndIndexSupplier.java
processing/src/main/java/org/apache/druid/segment/nested/ScalarLongColumnSerializer.java
processing/src/main/java/org/apache/druid/segment/nested/ScalarLongFieldColumnWriter.java
processing/src/main/java/org/a